# 1. Cargamos los datos de Pronostico ya predichos por el Modelo Machine Learning

In [2]:
# Leer el archivo Parquet en un DataFrame de pandas
df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

In [2]:
# df.tail()

### Filtramos los datos o registros para 1 solo caso de Optimizacion(porque LSTM, necesito 4 registros atras para predecir futuro)

In [4]:
datos_genetic = df.head(4)

# 2. Cargamos el Modelo de Machine Learning Entrenado

In [6]:
import tensorflow as tf
# Opcion 1
# from tensorflow.keras.models import load_model
# modelo_lstm = load_model('modelo_lstm_14_var.h5')

# Opcion 2
# # Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14_var.h5')

# 3. Construimos el Algoritmo Evolutivo

## PASO 1

### Función para generar los límites dinámicos basados en ±5 unidades 

In [7]:
import numpy as np
import pandas as pd
import numpy as np
import random

# Función para generar los límites dinámicos basados en ±5 unidades
def generate_dynamic_limits(row):
    """
    Genera X_min y X_max para una fila específica restando y sumando 5 unidades respectivamente.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    X_min = row - 2
    X_max = row + 2
    return X_min, X_max

## PASO 2

### Función para generar un individuo

In [8]:
# Función para generar un individuo
def generate_individual(random, df, start_index, n_var_independientes):
    """
    Genera un individuo basado en 4 filas consecutivas del DataFrame,
    usando los límites dinámicos ±2 para cada valor.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las 4 filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    
    Returns:
    individual (array): Array de tamaño (4, n_var_independientes) con los valores generados.
    """
    individual = []
    
    # Asegurarnos de trabajar solo con las primeras `n_var_independientes` columnas
    df_independientes = df.iloc[:, :n_var_independientes]
    
    for i in range(len(datos_genetic)):  # len(datos_genetic)  = 4 (numero de pasos atras que utilizaste para entrenar la LSTM)
        # Seleccionar la fila i (de start_index a start_index+3) con solo n_var_independientes columnas
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row)

        # Imprimir los límites generados para depuración
        print(f"Fila {i} - X_min: {X_min}")
        print(f"Fila {i} - X_max: {X_max}")

        # Generar una fila del individuo respetando los límites
        random_row = [
            random.uniform(X_min[j], X_max[j]) for j in range(len(row))
        ]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)

## Generar un individuo

In [19]:
# # Semilla para reproducibilidad
# random.seed(42)

# # Generar un individuo
# individuo = generate_individual(random, datos_genetic, start_index=0, n_var_independientes=14)
# individuo

## Paso 3 

### Función para generar la población mejorada con validaciones y depuración

In [10]:
# Función para generar la población mejorada con validaciones y depuración
def generate_population(n_individuos_poblacion, random, df, start_index, n_var_independientes):
    """
    Genera una población de individuos.
    
    Parameters:
    pop_size (int): Tamaño de la población(Numero de individuos para la poblacion)
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas.
    n_var_independientes (int): Número de variables independientes.
    
    Returns:
    list: Lista de individuos generados.
    """
    population = []
    
    for i in range(n_individuos_poblacion):
        # Generar un individuo
        individual = generate_individual(random, df, start_index, n_var_independientes)
        
        # Validar que el individuo se generó correctamente
        print(f"\nIndividuo {i+1}:")
        print(individual)
        
        # Añadir el individuo a la población
        population.append(individual)
    
    # Imprimir la forma de la población generada
    print("\nTamaño de la población generada:", len(population))
    print("Dimensiones de cada individuo:", np.array(population).shape)
    
    return population

In [20]:
# # Generar la población
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)
# poblacion

## Paso 4

### Funcion fitness, que ayude a evaluar los individuos generados

In [12]:
def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 4, 14)
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

In [21]:
# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")

### Codigo Final Consolidado

In [267]:
#x, y, z, efhcargado,efhvacio, distancia_ruta_pendiente_grados
# actual_gear (+- 1), roll

In [36]:
import pandas as pd
from random import Random
from time import time
from inspyred import ec
from random import Random
from time import time
import numpy as np

import tensorflow as tf

# # Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14_var.h5')

# Leer el archivo Parquet en un DataFrame de pandas
df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')
datos_genetic = df.head(4)

# Función para generar los límites dinámicos basados en ±5 unidades
def generate_dynamic_limits(row):
    """
    Genera X_min y X_max para una fila específica restando y sumando 5 unidades respectivamente.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    X_min = row - 2
    X_max = row + 2
    
    return X_min, X_max

# Función para generar un individuo
def generate_individual(random, df, start_index, n_var_independientes):
    """
    Genera un individuo basado en 4 filas consecutivas del DataFrame,
    usando los límites dinámicos ±2 para cada valor.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las 4 filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    
    Returns:
    individual (array): Array de tamaño (4, n_var_independientes) con los valores generados.
    """
    individual = []
    
    # Asegurarnos de trabajar solo con las primeras `n_var_independientes` columnas
    df_independientes = df.iloc[:, :n_var_independientes]
    
    for i in range(len(datos_genetic)):  # len(datos_genetic)  = 4 (numero de pasos atras que utilizaste para entrenar la LSTM)
        # Seleccionar la fila i (de start_index a start_index+3) con solo n_var_independientes columnas
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row)

        # Imprimir los límites generados para depuración
        #print(f"Fila {i} - X_min: {X_min}")
        #print(f"Fila {i} - X_max: {X_max}")

        # Generar una fila del individuo respetando los límites
        random_row = [
            random.uniform(X_min[j], X_max[j]) for j in range(len(row))
        ]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)

# individuo = generate_individual(random, datos_genetic, start_index=0, n_var_independientes=14)
# individuo

# Función para generar la población mejorada con validaciones y depuración
def generate_population(n_individuos_poblacion, random, df, start_index, n_var_independientes):
    """
    Genera una población de individuos.
    
    Parameters:
    pop_size (int): Tamaño de la población(Numero de individuos para la poblacion)
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas.
    n_var_independientes (int): Número de variables independientes.
    
    Returns:
    list: Lista de individuos generados.
    """
    population = []
    
    for i in range(n_individuos_poblacion):
        # Generar un individuo
        individual = generate_individual(random, df, start_index, n_var_independientes)
        
        # Validar que el individuo se generó correctamente
        print(f"\nIndividuo {i+1}:")
        print(individual)
        
        # Añadir el individuo a la población
        population.append(individual)
    
    # Imprimir la forma de la población generada
    print("\nTamaño de la población generada:", len(population))
    print("Dimensiones de cada individuo:", np.array(population).shape)
    
    return population

# Generar la población
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)
# poblacion

def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 4, 14)
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")\

def run_genetic_algorithm(modelo_lstm, df, n_individuos_poblacion=5, generations=5):
    # Crear el generador de números aleatorios
    prng = Random()
    #prng.seed(time())

    # Número de variables independientes en cada individuo
    n_var_independientes = 14  
    start_index = 0  # Índice de inicio para seleccionar las filas en tu dataset
    
    # # Inicializar la población
    # population = generate_population(n_individuos_poblacion, prng, df, start_index, n_var_independientes)
    
    # Crear el objeto del algoritmo genético (GA)
    ea = ec.GA(prng)

    # Configurar la función generadora de individuos
    def generator(random, args):
        return generate_individual(prng, df, start_index, n_var_independientes)
    
    # Crear un contenedor global para almacenar los fitness de cada generación
    fitness_history = {}

    #Configurar la función de evaluación (fitness)
    def evaluator(candidates, args):
        # Evaluar el fitness de todos los candidatos (individuos)
        fitness_values = fitness_function(candidates, args, modelo_lstm)
        
        # Obtener el número de generación actual
        current_generation = len(fitness_history) # + 1
        print(f"Evaluando Generación {current_generation}...")  # Imprimir para verificar
        
        # Guardar los fitness de la generación actual
        fitness_history[current_generation] = fitness_values
        
        # Mostrar los fitness de la generación actual
        print(f"\nFitness de la Generación {current_generation}: {fitness_values}")
        
        return fitness_values

    # Configuración de operadores genéticos
    ea.variator = [
        ec.variators.uniform_crossover,  # Cruce uniforme
        ec.variators.gaussian_mutation   # Mutación gaussiana
    ]

    # Cambiar el método de reemplazo a generational_replacement
    ea.replacer = ec.replacers.generational_replacement
    
    # ea.replacer = ec.replacers.steady_state_replacement  # Reemplazo en estado estable
    ea.selector = ec.selectors.tournament_selection  # Selección por torneo

    # Configurar la terminación por número de generaciones
    ea.terminator = ec.terminators.generation_termination


    # **Aquí agregamos el mensaje antes de la evolución**
    print(f"Ejecutando {generations} generaciones...")
    
    # Ejecutar la evolución
    final_pop = ea.evolve(
        generator=generator,
        evaluator=evaluator,
        pop_size=n_individuos_poblacion,
        maximize=False, #Problema de Minimizacion 
        num_elites=1,
        max_generations=generations
    )

        
    # Obtener el mejor individuo
    best_individual = min(final_pop, key=lambda x: x.fitness)
    best_fitness = best_individual.fitness

    print("\nMejor individuo (minimizar consumo de combustible):", best_individual.candidate)
    print("Consumo de combustible predicho (fitness):", best_fitness)

    return best_individual.candidate, best_fitness

# Asumiendo que tienes un modelo LSTM entrenado llamado `modelo_lstm`
best_individual, best_fitness = run_genetic_algorithm(modelo_lstm, datos_genetic, n_individuos_poblacion=5, generations=3)

Ejecutando 3 generaciones...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 0...

Fitness de la Generación 0: [19.575418, 181.35786, 29.671417, 40.29173, 336.7523]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 1...

Fitness de la Generación 1: [40.29173, 19.575418, 79.67817, 25.569904]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluando Generación 2...

Fitness de la Generación 2: [19.575418, 19.575418, 19.575418, 19.575418]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generac